# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [17]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [18]:
# read the cities.csv to create dataframe, previous cells are not needed to be executed as long as cities.csv was created
cities_data = pd.read_csv("output_data/cities.csv", encoding="utf-8")
cities_data.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,ushuaia,-54.80,-68.30,39.20,86,75,5.82,AR,1602816825
1,1,bluff,-46.60,168.33,55.00,60,100,10.00,NZ,1602816826
2,2,punta arenas,-53.15,-70.92,44.60,70,41,17.22,CL,1602816729
3,3,baykit,61.67,96.37,28.92,91,99,12.21,RU,1602816826
4,4,jiangyou,31.77,104.72,65.35,58,51,3.56,CN,1602816826


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [19]:
### Humidity Heatmap
gmaps.configure(api_key = g_key)
fig=gmaps.figure()

figure_layout = {'width' : '1000px', 'height' : '600px', 'border' : '1px solid black', 'padding' : '1px' }
locations = cities_data[['Lat','Lng']]
weights = cities_data['Humidity']
center_coordinates = (30.3333,31.2333)
fig = gmaps.figure(center = center_coordinates, map_type = "SATELLITE", zoom_level = 1.8, layout=figure_layout)



heatmap_layer = gmaps.heatmap_layer(locations,weights = weights, dissipating=False, max_intensity = 100, point_radius = 3)

fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', padding='1px', width='1000px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [20]:
clean_cities_data = cities_data.loc[(cities_data["Max Temp"] > 70) &
                                          (cities_data["Max Temp"] < 80) &
                                          (cities_data["Wind Speed"] < 10) &
                                          (cities_data["Cloudiness"] == 0)
                                          , :]
clean_cities_data = clean_cities_data.dropna(how='any')
clean_cities_data.reset_index(inplace=True)
del clean_cities_data['index']

# Display to output
clean_cities_data.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,62,san quintin,30.48,-115.95,72.97,58,0,4.12,MX,1602816841
1,147,tikapur,28.50,81.13,75.99,91,0,1.01,NP,1602816720
2,179,grand gaube,-20.01,57.66,72.00,80,0,1.99,MU,1602816867
3,222,camopi,3.17,-52.32,74.91,82,0,4.65,GF,1602816877
4,345,sambava,-14.27,50.17,72.00,82,0,9.06,MG,1602816904


In [21]:
hotel_df = clean_cities_data

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [22]:
hotel_df["Hotel Name"] = ""
hotel_df["Hotel Address"] = ""
hotel_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name,Hotel Address
0,62,san quintin,30.48,-115.95,72.97,58,0,4.12,MX,1602816841,,
1,147,tikapur,28.50,81.13,75.99,91,0,1.01,NP,1602816720,,
2,179,grand gaube,-20.01,57.66,72.00,80,0,1.99,MU,1602816867,,
3,222,camopi,3.17,-52.32,74.91,82,0,4.65,GF,1602816877,,
4,345,sambava,-14.27,50.17,72.00,82,0,9.06,MG,1602816904,,


In [27]:
params = {
    "radius": 5000,
    "keyword": "hotel",
    "key": g_key
}


# Use the lat/lng we recovered to identify airports
for index, row in clean_cities_data.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
#     print the name_address url, avoid doing for public github repos in order to avoid exposing key
#     print(name_address.url)

    # convert to json
    name_address = name_address.json()
    print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        clean_cities_data.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
        clean_cities_data.loc[index, "Hotel Address"] = name_address["results"][0]["vicinity"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

NameError: name 'json' is not defined

In [28]:
hotel_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name,Hotel Address
0,62,san quintin,30.48,-115.95,72.97,58,0,4.12,MX,1602816841,,
1,147,tikapur,28.50,81.13,75.99,91,0,1.01,NP,1602816720,,
2,179,grand gaube,-20.01,57.66,72.00,80,0,1.99,MU,1602816867,,
3,222,camopi,3.17,-52.32,74.91,82,0,4.65,GF,1602816877,,
4,345,sambava,-14.27,50.17,72.00,82,0,9.06,MG,1602816904,,
5,358,virginia beach,36.85,-75.98,71.01,81,0,1.99,US,1602816673,,
6,370,oriximina,-1.77,-55.87,79.57,67,0,3.15,BR,1602816910,,
7,379,bosaso,11.28,49.18,79.99,63,0,1.34,SO,1602816913,,
8,380,aswan,24.09,32.91,78.85,29,0,9.33,EG,1602816847,,
9,408,almeirim,-1.52,-52.58,78.13,77,0,6.38,BR,1602816846,,


In [29]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [30]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure(center=(0.0, 0.0), zoom_level=2)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)


# Display figure
fig

NameError: name 'heat_layer' is not defined